In [ ]:
import pandas as pd
import numpy as np
from typing import Dict, Any, List, Union, Optional
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
file_path = '/mnt/c/Users/Alexra/OneDrive/Documents/CASELOAD DATA/HC/RAW DATA'
output_path = '/mnt/c/Users/Alexra/OneDrive/Documents/CASELOAD DATA/ANALYSIS/HC'

# Data Cleaning

In [ ]:
def drop_nan_columns(df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    """
    Drop rows containing NaN values from the specified columns of a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame to process.
        columns (List[str]): A list of column names to check for NaN values.

    Returns:
        pd.DataFrame: The updated DataFrame with NaN-containing rows dropped.

    Raises:
        ValueError: If any of the specified columns are not present in the DataFrame.
    """
    # Validate that all specified columns exist in the DataFrame
    missing_columns = set(columns) - set(df.columns)
    if missing_columns:
        raise ValueError(f"Columns not found in DataFrame: {', '.join(missing_columns)}")

    # Identify columns with NaN values
    nan_columns = df[columns].columns[df[columns].isna().any()].tolist()

    # Log dropped rows if any
    if nan_columns:
        nan_count = df[columns].isna().sum()
        logger.info("Dropping rows with NaN values:")
        for col in nan_columns:
            logger.info(f"  {col}: {nan_count[col]} rows")

    # Drop rows with NaN values in specified columns
    original_row_count = len(df)
    df_cleaned = df.dropna(subset=columns)
    dropped_row_count = original_row_count - len(df_cleaned)

    if dropped_row_count > 0:
        logger.info(f"Total rows dropped: {dropped_row_count}")
    else:
        logger.info("No rows were dropped.")

    return df_cleaned

In [ ]:
def remove_duplicates(data: pd.DataFrame) -> pd.DataFrame:
    """
    Remove duplicates from a DataFrame.
    
    Args:
        data (pd.DataFrame): Input DataFrame.
        
    Returns:
        pd.DataFrame: DataFrame with duplicates removed.
    """
    num_duplicates = data.duplicated().sum()
    
    if num_duplicates > 0:
        logging.info(f"{num_duplicates} duplicates found.")
        data = data.drop_duplicates(keep="first").reset_index(drop=True)
        logging.info(f"{num_duplicates} duplicates removed.")
    else:
        logging.info("No duplicates found.")
    
    return data

In [ ]:
def create_date_column(df: pd.DataFrame, column_names: List[str], new_col: str) -> pd.DataFrame:
    """
    Creates a new date column in the DataFrame by concatenating the values of three specified columns.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        column_names (List[str]): A list of three column names to be concatenated [year, month, day].
        new_col (str): The name of the new date column to be created.

    Returns:
        pd.DataFrame: The DataFrame with the new date column added.

    Raises:
        ValueError: If the input list doesn't contain exactly three column names or if columns are missing.
    """
    if len(column_names) != 3:
        raise ValueError("column_names must contain exactly three elements: [year, month, day]")

    year_col, month_col, day_col = column_names

    # Check if all required columns exist in the DataFrame
    missing_columns = set(column_names) - set(df.columns)
    if missing_columns:
        raise ValueError(f"Missing columns in DataFrame: {', '.join(missing_columns)}")

    # Create copies to avoid SettingWithCopyWarning
    df = df.copy()

    try:
        # Convert year and day columns to integers
        df[year_col] = df[year_col].astype(float).astype(int)
        df[day_col] = df[day_col].astype(float).astype(int)

        # Concatenate the columns to create a date string
        df[new_col] = (df[year_col].astype(str) + '-' + 
                       df[month_col].astype(str) + '-' + 
                       df[day_col].astype(str))

        # Convert to datetime
        df[new_col] = pd.to_datetime(df[new_col], errors='coerce')

        # Log information about the conversion
        valid_dates = df[new_col].notna().sum()
        logger.info(f"Created new date column '{new_col}'. Valid dates: {valid_dates}/{len(df)}")

    except Exception as e:
        logger.error(f"Error creating date column: {str(e)}")
        raise

    return df

In [ ]:
def apply_title_case(text):
    """
    Apply title case to a given string.
    
    Args:
        text: The input string to process.
    
    Returns:
        str: The processed string in title case.
    """
    if pd.isna(text):
        return np.nan
    if not isinstance(text, str):
        logger.warning(f"Non-string value encountered: {text}")
        return str(text)
    return text.title()

In [ ]:
def process_outcome_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Process the 'outcome' column of the DataFrame by applying title case.
    
    Args:
        df (pd.DataFrame): The input DataFrame containing the 'outcome' column.
    
    Returns:
        pd.DataFrame: The DataFrame with the processed 'outcome' column.
    """
    if 'outcome' not in df.columns:
        logger.error("'outcome' column not found in the DataFrame")
        return df

    original_null_count = df['outcome'].isnull().sum()
    
    df['outcome'] = df['outcome'].apply(apply_title_case)
    
    new_null_count = df['outcome'].isnull().sum()
    if new_null_count > original_null_count:
        logger.warning(f"Number of null values in 'outcome' increased from {original_null_count} to {new_null_count}")
    
    non_string_count = df['outcome'].apply(lambda x: not isinstance(x, str) if pd.notna(x) else False).sum()
    if non_string_count > 0:
        logger.warning(f"Found {non_string_count} non-string values in 'outcome' after processing")

    return df

In [ ]:
def drop_null_values(df: pd.DataFrame, column_name: str = 'outcome') -> pd.DataFrame:
    """
    Drop rows from the DataFrame where the specified column contains null values.

    Args:
        df (pd.DataFrame): The DataFrame from which to drop rows.
        column_name (str): The name of the column to check for null values. Default is 'outcome'.

    Returns:
        pd.DataFrame: The DataFrame with rows containing null values in the specified column dropped.
    """
    initial_row_count: int = df.shape[0]
    cleaned_df: pd.DataFrame = df.dropna(subset=[column_name])
    final_row_count: int = cleaned_df.shape[0]
    dropped_row_count: int = initial_row_count - final_row_count
    
    logging.info(f"Total dropped rows with null values in '{column_name}': {dropped_row_count}")
    
    return cleaned_df

In [ ]:
def categorize_case(case_type: str, criminal_cases: Optional[List[str]]) -> str:
    """
    Categorize a case as 'Criminal' or 'Civil' based on its type.
    
    Args:
        case_type (str): The type of the case.
        criminal_cases (Optional[List[str]]): List of case types considered as criminal.
        
    Returns:
        str: 'Criminal' if the case type is in the criminal cases list or if criminal_cases is None, 'Civil' otherwise.
    """
    if criminal_cases is None:
        return 'Criminal'
    else:
        return 'Criminal' if case_type in criminal_cases else 'Civil'

In [ ]:

def categorize_cases(df: pd.DataFrame, criminal_cases: Optional[List[str]] = None) -> pd.DataFrame:
    """
    Categorize all cases in the DataFrame as 'Criminal' or 'Civil'.
    
    Args:
        df (pd.DataFrame): The input DataFrame containing case data.
        criminal_cases (Optional[List[str]]): List of case types considered as criminal.
            If None, all cases are categorized as 'Criminal'.
        
    Returns:
        pd.DataFrame: DataFrame with an added 'nature' column indicating case nature.
    """
    df['nature'] = df['case_type'].apply(lambda x: categorize_case(x, criminal_cases))

    # Check for presence of both case types
    if 'Criminal' not in df['nature'].values:
        logging.warning("No criminal cases found in the DataFrame.")
    if 'Civil' not in df['nature'].values:
        logging.warning("No civil cases found in the DataFrame.")
    
    return df

In [ ]:
def apply_dict(value: Any, dictionary: Dict[str, Union[str, List[Any]]]) -> Union[str, None]:
    """
    Find all keys in a dictionary where the given value matches.

    Args:
        value: The value to search for.
        dictionary: The dictionary to search in.

    Returns:
        A list of keys where the value matches, or None if no matches are found.
    """
    matching_keys = []
    for key, dict_value in dictionary.items():
        if isinstance(dict_value, str) and dict_value == value:
            matching_keys.append(key)
        elif isinstance(dict_value, list) and value in dict_value:
            matching_keys.append(key)
    
    if not matching_keys:
        return None
    elif len(matching_keys) == 1:
        return matching_keys[0]
    else:
        return matching_keys

In [ ]:
def generate_case_num(df: pd.DataFrame, court_col: str, caseid_type_col: str, caseid_no_col: str, filed_yyyy_col: str, new_col='case_number') -> pd.DataFrame:
    """
    Generates a case number by concatenating court, caseid_type, caseid_no, and filed_yyyy columns.

    Args:
        df (pd.DataFrame): The dataframe containing the necessary columns.
        court_col (str): The name of the column containing court information.
        caseid_type_col (str): The name of the column containing case ID type.
        caseid_no_col (str): The name of the column containing case ID number.
        filed_yyyy_col (str): The name of the column containing the year the case was filed.
        new_col (str): The name of the new column to be created for the case number. Default is 'case_num'.

    Returns:
        pd.DataFrame: DataFrame with the new case number column.
    """
    df[new_col] = df[court_col] + '/' + df[caseid_type_col] + '/' + df[caseid_no_col] + '/' + df[filed_yyyy_col].astype(str)
    return df

In [ ]:

def is_concluded(outcome: str, resolved_outcomes: List[str]) -> int:
    """
    Determine if the case is concluded based on the outcome.
    
    Args:
        outcome (str): The outcome of the case.
        resolved_outcomes (List[str]): List of outcomes considered as resolved.
        
    Returns:
        int: 1 if the case outcome is resolved, 0 otherwise.
    """
    return 1 if outcome in resolved_outcomes else 0

In [ ]:
def is_case_registered(outcome: str, activity_date: Union[pd.Timestamp, str], filed_date: Union[pd.Timestamp, str]) -> bool:
    """
    Determine if a case is registered based on its outcome and dates.

    Args:
        outcome (str): The outcome of the case.
        activity_date (Union[pd.Timestamp, str]): The date of the activity.
        filed_date (Union[pd.Timestamp, str]): The date the case was filed.

    Returns:
        bool: True if the case is registered, False otherwise.
    """
    try:
        # Normalize the outcome string
        normalized_outcome = outcome.strip().lower()

        # Check if the outcome indicates registration
        is_registered_outcome = 'case registered/filed' in normalized_outcome 

        # Convert dates to pd.Timestamp if they're strings
        if isinstance(activity_date, str):
            activity_date = pd.to_datetime(activity_date, errors='coerce')
        if isinstance(filed_date, str):
            filed_date = pd.to_datetime(filed_date, errors='coerce')

        # Check if dates are equal
        dates_match = pd.notna(activity_date) and pd.notna(filed_date) and activity_date == filed_date

        is_registered = is_registered_outcome and dates_match

        if is_registered:
            logger.debug(f"Case registered: outcome='{outcome}', activity_date={activity_date}, filed_date={filed_date}")
        
        return is_registered

    except Exception as e:
        logger.error(f"Error in is_case_registered: {e}")
        return False


In [ ]:
def process_case_status(df: pd.DataFrame, resolved_outcome: List[str]) -> pd.DataFrame:
    """
    Process the DataFrame to add 'concluded' and 'registered' columns.

    Args:
        df (pd.DataFrame): The input DataFrame containing case information.

    Returns:
        pd.DataFrame: The DataFrame with added 'concluded' and 'registered' columns.

    Raises:
        ValueError: If required columns are missing from the DataFrame.
    """
    required_columns = ['outcome', 'activity_date', 'filed_date']
    missing_columns = set(required_columns) - set(df.columns)
    if missing_columns:
        raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")

    #df['concluded'] = df['outcome'].apply(is_case_concluded)
    df['concluded'] = df['outcome'].apply(lambda x: is_concluded(x, resolved_outcome))
    df['registered'] = df.apply(lambda row: is_case_registered(row['outcome'], row['activity_date'], row['filed_date']), axis=1)

    logger.info(f"Processed {len(df)} cases")
    logger.info(f"Concluded cases: {df['concluded'].sum()}")
    logger.info(f"Registered cases: {df['registered'].sum()}")

    return df

In [ ]:
def analyze_court_outcomes(df: pd.DataFrame, start_date: str, end_date: str, outcome: str) -> pd.DataFrame:
    """
    Calculate the number of case outcomes per court within a specified period.
    
    Args:
        df (pd.DataFrame): A pandas DataFrame containing the data.
        start_date (str): The starting date of the period (YYYY-MM-DD format).
        end_date (str): The ending date of the period (YYYY-MM-DD format).
        outcome (str): A column representing the outcome of interest.
        
    Returns:
        pd.DataFrame: A DataFrame showing the number of resolved cases per court and case category.
    """
    try:
        period_start = pd.to_datetime(start_date)
        period_end = pd.to_datetime(end_date)
        
        if period_start > period_end:
            raise ValueError("start_date must be earlier than end_date")
        
        required_columns = {'court', 'broad_case_type', 'activity_date', outcome}
        if not required_columns.issubset(df.columns):
            missing_columns = required_columns - set(df.columns)
            raise KeyError(f"Missing required columns: {missing_columns}")
        
        filtered_cases = df[
            (df['activity_date'] >= period_start) &
            (df['activity_date'] <= period_end) &
            (df[outcome] == 1)
        ]
        
        if filtered_cases.empty:
            logging.warning("No cases found for the given date range and outcome.")
   
        outcome_by_type = (
            filtered_cases
            .groupby(['court', 'broad_case_type'])
            .size()
            .reset_index(name='num_cases')
        )

        result = outcome_by_type.pivot_table(
            index='court', 
            columns='broad_case_type', 
            values='num_cases', 
            fill_value=0
        )
        
        logging.info("Successfully calculated case outcomes per court.")
        return result
    
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        raise

In [ ]:
def check_time_limit(age, case_category, concluded, time_lines):
    """
    Check if a case falls within the specified time limit for its category and is concluded.
    
    Parameters:
        age (int): The age of the case in days.
        case_category (str): The category of the case.
        concluded (int): The status of the case conclusion (1 for concluded, 0 otherwise).
        time_lines (dict): A dictionary with case categories as keys and time limits as values.
        
    Returns:
        int: 1 if the case is within the time limit and concluded, otherwise 0.
    """
    time_limit = time_lines.get(case_category, 0)
    return 1 if age <= time_limit and concluded == 1 else 0


In [ ]:
def monthly_case_stats(df, registered_col, concluded_col):
    """Calculates monthly statistics for registered and concluded cases.

    Args:
        df (pandas.DataFrame): The input DataFrame containing case data.
        registered_col (str): The name of the column containing registered cases.
        concluded_col (str): The name of the column containing concluded cases.

    Returns:
        pandas.DataFrame: A DataFrame with monthly statistics for registered and concluded cases.
    """

    monthly_cases = df.groupby(['court', 'date_mon', 'case_type']).agg(
        registered=(registered_col, 'sum'),
        concluded=(concluded_col, 'sum')
    ).reset_index()

    return monthly_cases

In [ ]:
def add_timely_pmmu_column(data_f, time_lines):
    data_f['timely'] = data_f.apply(lambda row: True if row['broad_case_type'] in time_lines and row['concluded'] == 1 and row['age'] <= time_lines[row['broad_case_type']] else False, axis=1)
    return data_f

In [ ]:
def add_adjournment_columns(df, non_adjourned):
    """Adds 'adjourned' and 'adjournable' columns to the DataFrame."""
    df['adjourned'] = (df['reason_adj'].notnull() & df['comingfor'].apply(lambda x: x not in non_adjourned)).astype(int)
    df['adjournable'] = df['comingfor'].apply(lambda x: x not in non_adjourned).astype(int)
    return df

In [ ]:
def calculate_adjourned_per_court(df):
    """Calculates adjourned cases per court and reason."""
    return df.groupby(['court', 'reason_adj'])['adjourned'].sum().reset_index(name='count')



In [ ]:
def calculate_adjournment_proportion(df):
    """Calculates total adjourned, adjournable cases, and adjournment proportion per court."""
    court_summary = df.groupby('court').agg(
        total_adjourned=('adjourned', 'sum'),
        total_adjournable=('adjournable', 'sum')
    ).reset_index()
    court_summary['adjourn_proportion'] = court_summary['total_adjourned'] / court_summary['total_adjournable']
    return court_summary

In [ ]:
NON_ADJOUNABLE = [
    'Taxation and Issuance of Certificates',
    'Orders',
    'Appointments of  Mediator',
    'Screening of files for Mediation',
    'Post-judgment',
    'Re-activation',
    'Reactivation',
    'Notice of Taxation',
    'Entering Interlocutory Judgments',
    'Approval by DR', 
    'Registration/Filing-Application', 
    'Registration/Filing', 
    'Registration/Filing-Application',
 ]
RESOLVED_OUTCOMES = [
    'Ruling Delivered- Case Closed', 'Grant Confirmed', 'Matter Withdrawn',
    'Dismissed For Want Of Prosecution - Case Closed', 'Dismissed',
    'Terminated/ Struck Out/ Dismissed/case Closed', 'Judgment Delivered- Case Closed',
    'Application Allowed - Case Closed',
    'Matter Settled- Case Closed', 'Consent Recorded - Case Closed',
    'Judgment Delivered', 'Judgment Delivered- Acquittal',
    'Judgment Delivered- Convicted', 'Application Withdrawn - Case Closed',
    'Struck Out', 'Application Dismissed - Case Closed',
    'Out Of Court Settlement Reached', 'Terminated',
    'Ruling Delivered- Application Closed', 'Consolidated- Case Closed',
    'Interlocutory Judgement Entered', 'Abated', 'Limited Grant Issued',
    'Grant Revoked', 'Placed In Probation', 'Ruling Delivered- Accused Discharged',
    'Revision Declined', 'Retrial', 'Probation Orders Issued',
    'Matter Settled Through Mediation', 'Appeal Dismissed', 'Appeal Rejected', 
    'Order Issued - Case Closed',
    'Terminated',
    'File Transferred',
    'Terminated/ Struck Out/ Dismissed/Case Closed', 
    'Application Dismissed',
    'Application Allowed',
    'Terminated/ Struck Out/ Dismissed/Case Closed '
]
CRIMINAL_CASES = [
    'Murder Case',
    'Criminal Revision',
    'Criminal Appeal',
    'Murder - Gender Justice Criminal Case',
    'Criminal Court Martial Appeal',
    'Anti-Corruption and Economic Crimes Revision',
    'Criminal Miscellaneous Application',
    'Criminal Applications', 
    'COA Criminal Appeal'
]
BROAD_CASE_TYPES = {
    'Civil Matter': [
        'Civil Suit',
        'Anti-Corruption and Economic Crimes Suit',
        'Family Originating Summons',
        'Family Civil Case',
        'HCC(OS) Family',
        'Commercial Admiralty',
        'Commercial Matters',
    ],
    
    'Adoption': 'Family Adoption',
    'Divorce': 'Family Divorce Cause',

    'Criminal Application': 'Criminal Miscellaneous Application',

    'Civil Application': [
        'Civil Case Miscellaneous',
        'Judicial Review Miscellaneous',
        'JR  Petition Miscellaneous',
        'Anti-Corruption and Economic Crimes Miscellaneous',
        'Commercial Miscellaneous',
        'Constitution and Human Rights Petitions Miscellaneous',
        'Family Miscellaneous',
        'Commercial Arbitration',
    ],
    'Judicial Review': [
        'Anti-Corruption and Economic Crime Judicial review',
        'Judicial Review ELC',
        'Judicial Review',
    ],
    'Criminal Revision': [
        'Criminal Revision',
        'Anti-Corruption and Economic Crimes Revision',
    ],
    'Criminal Appeal': [
        'Criminal Appeal',
        'Criminal Court Martial Appeal',
        'Anti-Corruption and Economic Crimes Appeal',
    ],
    'Civil Appeal': [
        'Family Appeal',
        'Civil Appeal',
        'Commercial Appeal',
        'Constitution and Human Rights Election Petition Appeal',
        'Constitution and Human Rights Petition Appeal',
        'Constitution and Human Rights Election Petition Appeal',
        'Gender Justice Civil Appeal',
        'Constitution and Human Rights Miscellaneous Election Petition Appeal (MEPA)',
    ],
    'Constitution Petition': [
        'Anti Corruption and Economic Crimes Petition',
        'High Court Criminal Petition',
        'Constitution and Human Rights Petition (Civil)',
        'Constitution and Human Rights Election Petition',
        'High Court Constitution and Human Rights Petitions (Criminal)',
        'Commercial Petition',
    ],
    'Probate Administration': [
        'Family P&A Intestate',
        'Family P&A Ad Litem',
        'Family P&A Ad Colligenda',
        'Family P&A Citation',
        'Family P&A Testate',
        'Family P&A Resealing of Grant',
        'Family P&A De Bonis Non',
        'Resealing of Grant',
        'Citation-Family',
    ],
    'Murder': [
        'Murder Case',
        'Murder - Gender Justice Criminal Case',
    ],
    'Tax Appeal': [
        'Commercial Income Tax Appeal',
        'Commercial Custom Tax Appeal',
    ],
    'Bankruptcy and Insolvency' : [
        'Commercial Insolvency Notice Petition',
        'Commercial Insolvency Petition',
        'Commercial Bankruptcy Notice',
        'Commercial Insolvency Cause',
        'Commercial Insolvency Notice',
        'Commercial Bankruptcy Cause',
        'Commercial Winding Up Cause',
    ]
}

MERIT_CATEGORY ={
    'Merit Resolution' : [     
    'Judgment Delivered- Case Closed',
    'Judgment Delivered', 
    'Judgment Delivered- Acquittal',
    'Judgment Delivered- Convicted',
    'Retrial',
    'Appeal Dismissed',
    'Grant Revoked',
],
'Non Merit Resolution': [
    'Grant Confirmed', 
    'Matter Withdrawn',
    'Dismissed For Want Of Prosecution - Case Closed',
    'Dismissed',
    'Terminated/ Struck Out/ Dismissed/case Closed', 
    'Application Allowed - Case Closed',
    'Matter Settled- Case Closed', 
    'Consent Recorded - Case Closed',
    'Application Withdrawn - Case Closed',
    'Struck Out', 
    'Application Dismissed - Case Closed',
    'Out Of Court Settlement Reached', 
    'Terminated',
    'Consolidated- Case Closed',
    'Interlocutory Judgement Entered', 
    'Abated', 
    'Limited Grant Issued',
    'Placed In Probation', 
    'Revision Declined',  
    'Probation Orders Issued',
    'Matter Settled Through Mediation', 
    'Appeal Rejected', 
    'Order Issued - Case Closed',
    'Terminated'  
    ],
    'Rulings': [
    'Ruling delivered- Accused put on defense',
    'Ruling delivered',
    'Ruling Delivered- Case Closed', 
    'Ruling Delivered- Accused Discharged',
    'Ruling Delivered- Application Closed',
    'Ruling Delivered- Case Closed',
    ]
} 


productivity = {
    'Judgment Delivered': [
        'Judgment Delivered- Case Closed',
        'Judgment Delivered',
        'Judgment Delivered- Acquittal',
        'Judgment Delivered- Convicted',
        'Grant Revoked',
        'Retrial',
        'Ruling Delivered- Accused Discharged',
        ],

    'Ruling Delivered': [
        'Ruling delivered- Accused put on defense',
        'Ruling delivered'
        ],

    'Ruling Case Closed': [
        'Ruling Delivered- Case Closed', 
        ],
    'Final Grant': [
        'Grant Confirmed',
        'Limited Grant Issued',
        ],
    'Case Withdrawn': [
        'Matter Withdrawn',
        'Application Withdrawn - Case Closed',
        ],
   'Out Of Court Settlement': [
        'Consent Recorded - Case Closed',
        'Matter Settled Through Mediation',
        'Out Of Court Settlement Reached',
    ],
    'Dismissed':[
        'Dismissed For Want Of Prosecution - Case Closed',
        'Dismissed',
        'Appeal Dismissed',
        'Terminated',
        'Terminated/ Struck Out/ Dismissed/case Closed',
    ],
    'Case Transfered': [
        'File Transfered -case Closed',
        'File Transferred',
    ],
    'Case Closed': [
        'Struck Out',
        'Application Dismissed - Case Closed',
        'Application Allowed - Case Closed',
        'Matter Settled- Case Closed',
        'Ruling Delivered- Application Closed',
        'Consolidated- Case Closed',
        'Abated',
        'Placed In Probation',
        'Revision Declined',
        'Probation Orders Issued',
        'Appeal Rejected',
        'Interlocutory Judgement Entered',
    ],
}
HC_PMMU_TIME_LINES = {
    'Murder': 365, 
    'Constitution Petition': 180, 
    'Criminal Revision': 90, 
    'Judicial Review': 180, 
    'Civil Matter': 365,
    'Civil Appeal': 180,
    'Criminal Appeal': 180
}
# transferred
TRANSFERED_CASES = ['File Transfered -case Closed', 
        'File Transferred',]

In [ ]:

merit_classification = {
    'judgement': [
        'Judgment Delivered- Case Closed',
        'Judgment Delivered',
        'Judgment Delivered- Acquittal',
        'Judgment Delivered- Convicted',
        'Grant Revoked',
        'Retrial'
        ],

    'ruling_closed': [
        'Ruling Delivered- Case Closed', 
        'Ruling Delivered- Case Closed',
        'Ruling Delivered- Accused Discharged'
        ],

    'ruling_delivered': [
        'Ruling Delivered- Accused Put On Defense',
        'Ruling Delivered'
        ],

    'other_resolution': [
        'Grant Confirmed',
        'Limited Grant Issued',
        'Matter Withdrawn',
        'Application Withdrawn - Case Closed',
        'Consent Recorded - Case Closed',
        'Matter Settled Through Mediation',
        'Out Of Court Settlement Reached',
        'Dismissed For Want Of Prosecution - Case Closed',
        'Dismissed',
        'Appeal Dismissed',
        'Terminated',
        'Terminated/ Struck Out/ Dismissed/case Closed',
        'Struck Out',
        'Application Dismissed - Case Closed',
        'Application Allowed - Case Closed',
        'Matter Settled- Case Closed',
        'Ruling Delivered- Application Closed',
        'Consolidated- Case Closed',
        'Abated',
        'Placed In Probation',
        'Revision Declined',
        'Probation Orders Issued',
        'Appeal Rejected',
        'Interlocutory Judgement Entered',
    ],
}

In [ ]:
df = pd.read_csv(f'{file_path}/hc-q3-24-25.csv')

In [ ]:
df['outcome'] = df['outcome'].replace('Terminated/ Struck Out/ Dismissed/Case Closed ', 'Terminated')

In [ ]:
judges_df = pd.read_csv(f'{file_path}/final_judges.csv')

In [ ]:
# keep if the df['judge_1'] is in the judges_df 
df = df[df['judge_1'].isin(judges_df['name'])].copy()

In [ ]:
df = create_date_column(df.copy(), ['date_dd', 'date_mon', 'date_yyyy'], 'activity_date')
df = create_date_column(df.copy(), ['filed_dd', 'filed_mon', 'filed_yyyy'], 'filed_date')
df = drop_nan_columns(df, ['date_dd', 'date_mon', 'date_yyyy', 'caseid_type', 'caseid_no',
       'filed_dd', 'filed_mon', 'filed_yyyy', 'case_type', 'comingfor'])
df = remove_duplicates(df)
df = df.apply(lambda x: x.strip().title() if isinstance(x, str) else x)
df = process_outcome_column(df)
df = drop_null_values(df)
df = categorize_cases(df, CRIMINAL_CASES)
df['broad_case_type'] = df['case_type'].apply(lambda x: apply_dict(x, BROAD_CASE_TYPES))
df = generate_case_num(df, 'court', 'caseid_type', 'caseid_no', 'filed_yyyy')
df['age'] = (df['activity_date'] - df['filed_date']).dt.days
df = process_case_status(df, RESOLVED_OUTCOMES)
df['merit_category'] = df['outcome'].apply(lambda x: apply_dict(x, MERIT_CATEGORY))
df = add_timely_pmmu_column(df, HC_PMMU_TIME_LINES)
df['comingfor'] = df['comingfor'].str.strip()
df = add_adjournment_columns(df, NON_ADJOUNABLE)
df['transfer'] = df['outcome'].apply(lambda x: x in TRANSFERED_CASES)

In [ ]:
start_date = pd.to_datetime('2024-07-01')
end_date = pd.to_datetime('2024-12-31')  

# Ensure the activity_date column is in datetime format
df['activity_date'] = pd.to_datetime(df['activity_date'], errors='coerce')

# Filter the DataFrame based on the date range
df = df[(df['activity_date'] >= start_date) & (df['activity_date'] <= end_date)]

In [ ]:
df_concluded = df[df['concluded'] == 1]

In [ ]:
clean_df = df.drop_duplicates(subset=['case_number'], keep='first')

In [ ]:
df['classification'] = df['outcome'].apply(lambda x: apply_dict(x, merit_classification))

In [ ]:
df.pivot_table(index='judge_1', columns='classification', values='concluded', aggfunc='sum', fill_value=0).rename_axis(columns=None).to_csv(f'{output_path}/classification.csv', index=True)

In [ ]:
if 'classification' in df.columns:
	df['ruling_delivered'] = (df['classification'] == 'ruling_delivered').astype(int)
else:
	raise KeyError("'classification' column is not defined in the DataFrame.")

In [ ]:
df.groupby('judge_1')['ruling_delivered'].sum().reset_index(name='total').to_csv(f'{output_path}/ruling_delivered.csv', index=False)

In [ ]:

# Extract the quarter from the activity_date column
df['quarter'] = df['activity_date'].dt.to_period('Q')

In [ ]:
df['resolution_mode'] = df['outcome'].apply(lambda x: apply_dict(x, merit_classification))

In [ ]:
filed_cases = analyze_court_outcomes(df, start_date, end_date, 'registered')
resolved_cases = analyze_court_outcomes(df, start_date, end_date, 'concluded')
monthly_filed_cases = df.groupby(['court','date_mon']).agg({'registered':'sum'}).reset_index()
monthly_concluded_cases = df.groupby(['court','date_mon']).agg({'concluded':'sum'}).reset_index()
average_time_to_conclude = df.loc[df['concluded'] == 1].pivot_table(index='court', columns='nature', values='age', aggfunc='mean', fill_value=0).round(2)
monthly_stats = monthly_case_stats(df, 'registered', 'concluded')
court_productivity = df.pivot_table(index='court', columns='merit_category', values='concluded', aggfunc='sum', fill_value=0).rename_axis(columns=None)
total_resolved_cases = df.pivot_table( index='court', columns='broad_case_type', values='timely', aggfunc=lambda x: np.sum(df.loc[x.index, 'concluded'] == 1), fill_value=0)
timely_resolved_cases = df[df['timely']].pivot_table(index='court', columns='broad_case_type', values='timely', aggfunc='sum', fill_value=0)
percentage_timely = (timely_resolved_cases / resolved_cases ).fillna(0)
adjourned_per_court = calculate_adjourned_per_court(df)
adjourned_proportion = calculate_adjournment_proportion(df)
transfered_cases = df[df['transfer']].pivot_table(index='court', columns='broad_case_type', values='transfer', aggfunc='sum', fill_value=0)

In [ ]:
column_order = ['Murder', 'Criminal Appeal', 'Criminal Application', 
                'Criminal Revision', 'Civil Matter', 'Civil Appeal', 'Civil Application', 
                'Constitution Petition', 'Judicial Review',
                  'Bankruptcy and Insolvency', 'Tax Appeal', 
                  'Adoption', 'Divorce', 'Probate Administration']

In [ ]:


resolved_cases.to_csv(f'{output_path}/{end_date}_resolved_cases.csv', columns=column_order, index=True)
monthly_filed_cases.to_csv(f'{output_path}/{end_date}_filed_per_month.csv', index=False)
monthly_concluded_cases.to_csv(f'{output_path}/{end_date}_monthly_concluded.csv', index=False)
court_productivity.to_csv(f'{output_path}/{end_date}_court_productivity.csv')
average_time_to_conclude.to_csv(f'{output_path}/{end_date}_average_time_to_conclude.csv')
percentage_timely.to_csv(f'{output_path}/{end_date}_resolved_within_timeline.csv')
adjourned_per_court.to_csv(f'{output_path}/{end_date}_adjournement.csv', index=False)
adjourned_proportion.to_csv(f'{output_path}/{end_date}_adjourned_proportion.csv', index=False)
transfered_cases.to_csv(f'{output_path}/{end_date}_transfered_cases.csv',columns=column_order, index=True)

In [ ]:
column_order = ['Murder', 'Criminal Appeal', 'Criminal Application', 
                'Criminal Revision', 'Civil Matter', 'Civil Appeal', 'Civil Application', 
                'Constitution Petition', 'Judicial Review',
                  'Bankruptcy and Insolvency', 'Tax Appeal',
                  'Adoption', 'Probate Administration']
filed_cases.to_csv(f'{output_path}/{end_date}_filed_cases.csv', columns=column_order, index=True)

In [ ]:
def categorize_concluded_cases(df):
    """
    Categorize concluded cases into '1-3 year' and 'over 3 year' based on age column.

    Args:
        df (pandas.DataFrame): DataFrame containing the data.

    Returns:
        pandas.DataFrame: DataFrame with an additional column 'age_category'.
    """
    # Filter only concluded cases
    df_concluded = df[df['concluded'] == 1]

    # Calculate age in years
    df_concluded['age_years'] = df_concluded['age'] / 365

    # Categorize based on age
    df_concluded.loc[:, 'age_category'] = df_concluded['age_years'].apply(lambda x: '1-3 year' if 1 <= x <= 3 else 'over 3 year')

    # Merge back to original DataFrame
    df = pd.merge(df, df_concluded[['case_number', 'age_category']], on='case_number', how='left')

    return df


In [ ]:
def track_case_transitions(df, start_period, end_period):
    """
    Track case transitions to over 1 year and over 3 years based on age.

    Args:
        df (pd.DataFrame): DataFrame containing case data with an 'age' column.
        start_period (pd.Timestamp): The start period as a timestamp.
        end_period (pd.Timestamp): The end period as a timestamp.

    Returns:
        pd.DataFrame: DataFrame with additional columns indicating transitions.
    """
    # Calculate the age difference between the periods
    age_difference = (end_period - start_period).days

    # Define thresholds in days
    one_year_days = 365
    three_year_days = 3 * one_year_days

    # Transition to 1 year category
    df['transition_to_1_year'] = (
        (df['age'] < one_year_days) &
        (df['age'] + age_difference >= one_year_days) &
        (df['age'] + age_difference < 2 * one_year_days)
    ).astype(int)

    # Transition to 3 year category
    df['transition_to_3_year'] = (
        (df['age'] < three_year_days) &
        (df['age'] + age_difference >= three_year_days) &
        (df['age'] + age_difference < 4 * one_year_days)
    ).astype(int)

    return df

In [ ]:
df  = track_case_transitions(df, start_date, end_date)

In [ ]:
# max transition_to_1_year if df['concluded']== 1 to avoid double counting
df['transition_1_year'] = df.apply(lambda x: max(x['transition_to_1_year'], x['concluded']), axis=1)
df['transition_3_year'] = df.apply(lambda x: max(x['transition_to_3_year'], x['concluded']), axis=1)

In [ ]:
df_2 = df.copy()


In [ ]:
# Remove duplicates
df_2 = df_2.drop_duplicates(subset=['court', 'case_number'])

In [ ]:
df_2['age_2'] = (df_2['activity_date'] - end_date).dt.days

In [ ]:
transited_backlog = df_2.groupby('court').agg(
    transition_to_1_year=('transition_to_1_year', 'sum'),
    transition_to_3_year=('transition_to_3_year', 'sum')
).reset_index()

In [ ]:
transited_backlog.to_csv(f'{output_path}/{end_date}_transited_backlog.csv', index=False)

In [ ]:
concluded_backlog = df_2[df_2['concluded']==1].groupby('court').agg(
    transition_to_1_year=('transition_to_1_year', 'sum'),
    transition_to_3_year=('transition_to_3_year', 'sum')
).reset_index()

In [ ]:
# resolved_backlog_over 3 
resolved_backlog_over_3 = df_2[(df_2['concluded']==1) & (df['age_1'] > 1080)].groupby('court').agg(
    transition_to_3_year=('transition_to_3_year', 'sum')
).reset_index()
resolved_backlog_over_3.to_csv(f'{output_path}/{end_date}_resolved_backlog_over_3.csv', index=False)

In [ ]:
concluded_backlog.to_csv(f'{output_path}/{end_date}_concluded_backlog.csv', index=False)

In [ ]:
df.groupby('transition_to_1_year')['case_number'].count()

In [ ]:
df_2.query('court == "Voi" & concluded == 1 & transition_to_1_year == 1')
#.groupby('case_number')['transition_to_1_year'].count()

In [ ]:
df.query('court == "Garsen" & case_number == "Garsen/HCCA/E002/2024"')

In [ ]:
# create a table where df['concluded] == 1 and transited to df['transition_to_1_year'] and df['transition_to_3_year'] per court
df[df['concluded']== 1].groupby('court')['transition_to_1_year'].sum().reset_index().sort_values(by='transition_to_1_year', ascending=False)

In [ ]:
df = categorize_concluded_cases(df)

In [ ]:
df['resolved'] = df.groupby('case_number')['concluded'].transform('max')
pending_df = df[df['resolved'] != 1]
# custody_df.groupby(['court', 'case_type'])['reg'].count().reset_index(name='count')
# custody_df.groupby(['court','case_number'])['reg'].count().sort_values(ascending=False).reset_index(name='count').to_csv(f'{file_path}/hc_custody_cases.csv', index=False)
# custody_df.to_csv(f'{file_path}/hc_custody_data.csv', index=False)

In [ ]:
pending_df.to_csv(f'{file_path}/kibera_pending.csv', index=False)

In [ ]:
filed_df = df[df['registered'] == 1]
male_applicants = filed_df[(filed_df['male_applicant'] == 1) & (filed_df['case_category'] != 'murder')].groupby('case_category').size().reset_index(name='count')
female_applicants = filed_df[(filed_df['female_applicant'] == 1) & (filed_df['case_category'] != 'murder')].groupby('case_category').size().reset_index(name='count')
male_accused = filed_df[(filed_df['male_defendant'] == 1) & (filed_df['case_category'] == 'murder')].groupby('case_category').size().reset_index(name='count')
male_murder_defendants = filed_df[(filed_df['male_defendant'] == 1) & (filed_df['case_category'] == 'murder')].groupby('case_category').size().reset_index(name='count')
female_murder_defendants = filed_df[(filed_df['female_defendant'] == 1) & (filed_df['case_category'] == 'murder')].groupby('case_category').size().reset_index(name='count')
applicants = pd.merge(female_applicants, male_applicants, on='case_category', how='outer', suffixes=('_female', '_male'))
applicants.to_csv(f'{output_path}/applicants_gender.csv', index=False)



## Kisumu analysis

In [ ]:
# kisumu_df['filed_date'] = pd.to_datetime(kisumu_df['filed_dd'].astype(str) + '-' + kisumu_df['filed_mon'].astype(str) + '-' + kisumu_df['filed_yyyy'].astype(str), format='%d-%b-%Y')
# kisumu_df = drop_nan_columns(kisumu_df, ['date_dd', 'date_mon', 'date_yyyy', 'caseid_type', 'caseid_no',
#        'filed_dd', 'filed_mon', 'filed_yyyy', 'case_type', 'comingfor'])
# kisumu_df = kisumu_df.apply(lambda x: x.strip().title() if isinstance(x, str) else x)
# kisumu_df = process_outcome_column(kisumu_df)
# kisumu_df = drop_null_values(kisumu_df)
# kisumu_df['broad_case_type'] = kisumu_df['case_type'].apply(lambda x: apply_dict(x, BROAD_CASE_TYPES))

# Milimani divs

In [ ]:
df = pd.read_csv(f'{file_path}/processed_df.csv')

In [ ]:
milimani_df = df[df['court'].str.contains('Milimani')]


In [ ]:
milimani_df_2 = df[df['court'].str.contains('Milimani')]


In [ ]:
milimani_df_2.columns

In [ ]:
['court', 'date_dd', 'date_mon', 'date_yyyy', 'caseid_type', 'caseid_no',
       'filed_dd', 'filed_mon', 'filed_yyyy', 'original_court',
       'original_code', 'original_number', 'original_year', 'case_type',
       'judge_1', 'judge_2', 'judge_3', 'judge_4', 'judge_5', 'judge_6',
       'judge_7', 'comingfor', 'outcome', 'reason_adj', 'next_dd', 'next_mon',
       'next_yyyy', 'male_applicant', 'female_applicant',
       'organization_applicant', 'male_defendant', 'female_defendant',
       'organization_defendant', 'legalrep', 'applicant_witness',
       'defendant_witness', 'custody', 'other_details', 'activity_date',
       'filed_date', 'nature', 'broad_case_type', 'case_number',
       'concluded', 'registered', 'productivity', 'age', 'time_lines',
       'adjourned', 'adjournable', 'set_date', 'delivery_date',
       'time_taken_days', 'judgment_status', 'delivery_category'],

In [ ]:
combined_df = pd.concat([milimani_df, milimani_df_2])

In [ ]:
combined_df = combined_df[['court', 'date_dd', 'date_mon', 'date_yyyy', 'caseid_type', 'caseid_no',
       'filed_dd', 'filed_mon', 'filed_yyyy', 'original_court',
       'original_code', 'original_number', 'original_year', 'case_type',
       'judge_1', 'judge_2', 'judge_3', 'judge_4', 'judge_5', 'judge_6',
       'judge_7', 'comingfor', 'outcome', 'reason_adj', 'next_dd', 'next_mon',
       'next_yyyy', 'male_applicant', 'female_applicant',
       'organization_applicant', 'male_defendant', 'female_defendant',
       'organization_defendant', 'legalrep', 'applicant_witness',
       'defendant_witness', 'custody', 'other_details', 'activity_date',
       'filed_date', 'nature', 'broad_case_type', 'case_number', 'concluded',
       'registered', 'age', 'adjourned',
       'adjournable',]]

In [ ]:
df = pd.read_csv(f'{file_path}/milimani_data.csv')

In [ ]:
# Convert the activity_date column to datetime format
df['activity_date'] = pd.to_datetime(df['activity_date'], format='mixed')

# Filter only concluded cases
concluded_cases = df[df['concluded'] == 1]

# Extract the quarter from the activity_date column
concluded_cases['quarter'] = concluded_cases['activity_date'].dt.to_period('Q')


In [ ]:
# Filter only concluded cases
registered_cases = df[df['registered'] == 1]

# Extract the quarter from the activity_date column
registered_cases['quarter'] = registered_cases['activity_date'].dt.to_period('Q')


In [ ]:
quarterly_registered_cases = registered_cases.groupby('quarter').size().reset_index(name='filed_cases_count')

In [ ]:
# Group by the quarter and count the number of concluded cases
quarterly_concluded_cases = concluded_cases.groupby('quarter').size().reset_index(name='concluded_cases_count')

In [ ]:
# combine the two DataFrames
quarterly_cases = pd.merge(quarterly_registered_cases, quarterly_concluded_cases, on='quarter', how='outer')

In [ ]:
quarterly_cases.to_csv(f'{file_path}/quarterly_cases.csv', index=False)